In [ ]:
from neo4j import GraphDatabase

In [ ]:
URI = "neo4j://localhost"
AUTH = ("neo4j", "neo4jneo4j")
DB = 'medical'

In [ ]:
DATASET= 'medical_2000' # medical_20

In [ ]:
CONSTRAINTS = [
    "CREATE CONSTRAINT PatientConstraint IF NOT EXISTS FOR (p:Patient) REQUIRE p.PatientID IS UNIQUE;",
    "CREATE CONSTRAINT DrugConstraint IF NOT EXISTS FOR (d:Drug) REQUIRE d.ID IS UNIQUE;",
    "CREATE CONSTRAINT DiagConstraint IF NOT EXISTS FOR (d:Diag) REQUIRE d.ID IS UNIQUE;",
    "CREATE CONSTRAINT WeightConstraint IF NOT EXISTS FOR (w:Weight) REQUIRE w.Value IS UNIQUE;",
    "CREATE CONSTRAINT IHDConstraint IF NOT EXISTS FOR (i:IHD) REQUIRE i.Name IS UNIQUE;"
]

In [ ]:
LOAD_CSVS = [
"""LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/neo4j-field/medical-gds-sandbox/main/data/""" + DATASET + """/patients.csv' AS line
MERGE (p:Patient {PatientID: line.PatientID})
SET p.Age = toInteger(line.Age)
MERGE (w:Weight {Value: toInteger(line.Weight)})
MERGE (p)-[:MEASURES]->(w)""",
"""LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/neo4j-field/medical-gds-sandbox/main/data/""" + DATASET + """/ihds.csv' AS line
MERGE (p:Patient {PatientID: line.PatientID})
MERGE (i:IHD {Name:line.IHD})
MERGE (p)-[:IHD_RISK]->(i)""",
"""LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/neo4j-field/medical-gds-sandbox/main/data/""" + DATASET + """/drugs.csv' AS line
MERGE (p:Patient {PatientID: line.PatientID})
MERGE (d:Drug {ID: line.Drug})
MERGE (p)-[:PRESCRIBED]->(d)""",
"""LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/neo4j-field/medical-gds-sandbox/main/data/""" + DATASET + """/diags.csv' AS line
MERGE (p:Patient {PatientID: line.PatientID})
MERGE (d:Diag {ID: line.Diag})
MERGE (p)-[:DIAGNOSED_WITH]->(d)"""
]

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver: 
    driver.verify_connectivity() 

In [ ]:
def delete_all(tx):  
    result = tx.run("MATCH (n) DETACH DELETE n")
    summary = result.consume()
    return summary

In [ ]:
with driver.session(database=DB) as session:
    summary = session.execute_write(delete_all)  # Delete all
    for c in CONSTRAINTS:
        session.execute_write(lambda tx: tx.run(c))
    for l in LOAD_CSVS:
        session.execute_write(lambda tx: tx.run(l))